In [4]:
%load_ext autoreload
%autoreload 2

from NLP_Lib import *

import tensorflow as tf
from tensorflow import keras
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import math
import numpy as np
import tensorflow_hub as hub
import tensorflow_text as text
import json

from official.nlp import optimization # for creating custom optimizer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
# Get the info of the model we want to load
PATH_TO_MODEL_INFO = os.path.join(
    PATH_TO_MODELS_DIRECTORY,
    "10",
    "24",
    "07:30:28")

# save_at_n.hf files are to big to push to github
PATH_TO_WEIGHTS_TO_LOAD = os.path.join(PATH_TO_MODEL_INFO, "save_at_11.h5")
PATH_TO_JSON_PARAMS = os.path.join(PATH_TO_MODEL_INFO, "training_params.json")

params_dict = json.load(open(PATH_TO_JSON_PARAMS))
print("Path to weights:", PATH_TO_WEIGHTS_TO_LOAD)

Path to weights: ./models/10/24/07:30:28/save_at_11.h5


In [16]:
# Using the fetched model params and weights, load the model we want to predict with
optimizer = optimization.create_optimizer(
    init_lr=float(params_dict[INITIAL_LEARNING_RATE]),
    num_train_steps=int(params_dict[NUMBER_OF_TRAINING_STEPS]),
    num_warmup_steps=int(params_dict[NUMBER_OF_WARMUP_STEPS]),
    optimizer_type=params_dict[OPTIMIZER])

model = build_classifier_model(SMALL_BERT[0], SMALL_BERT[1], optimizer)
model.load_weights(PATH_TO_WEIGHTS_TO_LOAD)

In [17]:
# Load in the testing data tsv, use the loaded model to make
# predictions and save them to a .csv in the form kaggle defines
testing_df = pd.read_table(PATH_TO_TESTING_TSV)
X = np.array(testing_df[TEXT_COL])
preds = model.predict(X)

submission_df = pd.DataFrame(dict(
        Id=testing_df[ID_COL],
        Category=map_probs_to_class(preds, map_idx_to_subclass)))

In [18]:
# use this to map a subclass predictions DF to a class predictions DF where all non-main
# Simpsons character predictions get changed to "Other"
submission_df[SUBMISSION_CATEGORY_COL] = submission_df[SUBMISSION_CATEGORY_COL].apply(lambda cls: cls if cls in CLASSES else OTHER)
submission_df

,Id,Category
0,1,Bart Simpson
1,2,Lisa Simpson
2,3,Homer Simpson
3,4,Bart Simpson
4,5,Lisa Simpson
...,...,...
10494,10520,Homer Simpson
10495,10521,Homer Simpson
10496,10522,Other
10497,10523,Bart Simpson


In [14]:
submission_df.to_csv(
    os.path.join(PATH_TO_MODEL_INFO, "kaggle_submission_e11.csv"),
    index=False) # ignore the row index column when writing to the csv